<a href="https://colab.research.google.com/github/tpals213/ict/blob/main/1_1_numpy_tensorflow_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1_1_numpy_tensorflow_pytorch.ipynb
구글 코랩을 이용한 코드 작성 첫번째 파일  
PyTorch 설치전 : 런타임 메뉴 > 런타임 유형 변경 > GPU 지정  
PyTorch 철치 (매 세션마다 설치 필요)  
*주피터 노트북의 설치는 !pip3 로 시작*

In [ ]:
!pip3 install torch
!pip3 install torchvision

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

# Framework Comparsion (패키지 비교)

* Numpy vs Tensorflow vs Pytorch
* output = x * y + z


In [ ]:
import numpy as np
from datetime import datetime

start = datetime.now()

np.random.seed(0) # 랜덤값 발생 고정시킴

N, D = 3, 4

x = np.random.randn(N, D) # 3행 4열 배열에 0 ~ 1 사이의 랜덤값 채움
y = np.random.randn(N, D)
z = np.random.randn(N, D)

a = x * y
b = a + z
c = np.sum(b)

grad_c = 1.0
grad_b = grad_c * np.ones((N, D))
grad_a = grad_b.copy()
grad_z = grad_b.copy()
grad_y = grad_a * y
grad_x = grad_a * y

print(grad_x)
print(grad_y)
print(grad_z)

print('연산 처리 시간 : ', datetime.now() - start)

[[ 0.76103773  0.12167502  0.44386323  0.33367433]
 [ 1.49407907 -0.20515826  0.3130677  -0.85409574]
 [-2.55298982  0.6536186   0.8644362  -0.74216502]]
[[ 0.76103773  0.12167502  0.44386323  0.33367433]
 [ 1.49407907 -0.20515826  0.3130677  -0.85409574]
 [-2.55298982  0.6536186   0.8644362  -0.74216502]]
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
연산 처리 시간 :  0:00:00.009609


In [ ]:
# import tensorflow as tf
import numpy as np
from datetime import datetime

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# tf.placeholder() 함수 AttributeError 발생 시 해결 방법  
tensorflow 2.0 에서는 사용 못 함  
```(python)
import tensorflow.compat.v1 as tf  
tf.disable_v2_behavior()
```


In [ ]:
start_tf = datetime.now()

# 실행 동작을 미리 정의해 놓음
with tf.device('/gpu:0'):    # 실행시 gpu 0 사용을 지정
  x = tf.placeholder(tf.float32)  # 나중에 값을 넣을 공간을 만듦
  y = tf.placeholder(tf.float32)
  z = tf.placeholder(tf.float32)

  a = x * y
  b = a + z
  c = tf.reduce_sum(b)

grad_x, grad_y, grad_z = tf.gradients(c, [x, y, z])

# 준비된 공간에 값을 채우는 설정
with tf.Session() as sess:
  values = {
      x: np.random.rand(N, D),
      y: np.random.rand(N, D),
      z: np.random.rand(N, D)
  }
  out = sess.run([c, grad_x, grad_y, grad_z], feed_dict=values)
  c_val, grad_x_val, grad_y_val, grad_z_val = out

print(grad_x_val)
print(grad_y_val)
print(grad_z_val)

print('연산 처리 시간 : ', datetime.now() - start_tf)

[[0.9755215  0.8558034  0.01171408 0.35997805]
 [0.72999054 0.17162968 0.5210366  0.05433799]
 [0.19999653 0.0185218  0.7936977  0.22392468]]
[[0.16249293 0.6155596  0.12381998 0.8480082 ]
 [0.807319   0.56910074 0.4071833  0.069167  ]
 [0.69742876 0.45354268 0.7220556  0.8663823 ]]
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
연산 처리 시간 :  0:00:00.049565


In [ ]:
import torch
from torch.autograd import Variable
from datetime import datetime

start = datetime.now()

N, D = 3, 4 # 생략 가능 : 위쪽 셀에서의 선언을 그대로 사용해도 됨

# 자동 미분 계산 클래스 : autograd.Variable
x = Variable(torch.randn(N, D).cuda(), requires_grad=True)
y = Variable(torch.randn(N, D).cuda(), requires_grad=True)
z = Variable(torch.randn(N, D).cuda(), requires_grad=True)

a = x * y
b = a + z
c = torch.sum(b)

# gradient(경사도) 자동 계산 수행
# 기울기(gradient)가 1.0 이라고 가정하고 최종 값인 c 에서
# backward를 통해서 역전파를 해 줌
# c.backward(gradient=torch.cuda.FloatTensor([1.0]))  # GPU 버전
c.backward(gradient=torch.tensor(1, dtype=torch.float))  # CPU 버전

print(x.grad)
print(y.grad)
print(z.grad)

print('연산 처리 시간 : ', datetime.now() - start)

tensor([[ 1.4596,  0.4071,  0.7562,  1.5897],
        [ 0.9210,  0.8245,  1.3401, -0.0881],
        [ 0.0053, -0.5832,  0.4553,  0.0296]], device='cuda:0')
tensor([[ 0.6925,  0.8331, -0.6427, -0.8713],
        [-0.6039,  1.6108, -0.0934, -1.2108],
        [-0.4190, -1.6256,  0.1264, -1.0214]], device='cuda:0')
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], device='cuda:0')
연산 처리 시간 :  0:00:00.393700
